In [3]:
from google.colab import drive
import json
drive.mount('/content/gdrive',force_remount=False)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
!cp /content/gdrive/MyDrive/converted_224x224.tar.gz /content
%cd /content


/content


In [ ]:
!tar -xvzf /content/converted_224x224.tar.gz

In [26]:
!ls /content/converted_224x224/

frame_000001.jpg  frame_053946.jpg  frame_105543.jpg  frame_152735.jpg
frame_000035.jpg  frame_053947.jpg  frame_105544.jpg  frame_152736.jpg
frame_000037.jpg  frame_053948.jpg  frame_105545.jpg  frame_152737.jpg
frame_000038.jpg  frame_053949.jpg  frame_105546.jpg  frame_152738.jpg
frame_000039.jpg  frame_053950.jpg  frame_105547.jpg  frame_152739.jpg
frame_000040.jpg  frame_053951.jpg  frame_105548.jpg  frame_152740.jpg
frame_000042.jpg  frame_053952.jpg  frame_105549.jpg  frame_152741.jpg
frame_000043.jpg  frame_053953.jpg  frame_105550.jpg  frame_152742.jpg
frame_000044.jpg  frame_053954.jpg  frame_105551.jpg  frame_152743.jpg
frame_000045.jpg  frame_053955.jpg  frame_105552.jpg  frame_152744.jpg
frame_000047.jpg  frame_053956.jpg  frame_105553.jpg  frame_152745.jpg
frame_000048.jpg  frame_053957.jpg  frame_105554.jpg  frame_152746.jpg
frame_000049.jpg  frame_053958.jpg  frame_105555.jpg  frame_152747.jpg
frame_000050.jpg  frame_053959.jpg  frame_105556.jpg  frame_152748.jpg
frame_

In [7]:
import os,inspect
import sys
import time

import numpy as np
import pandas as pd

import pdb
import gc
from tqdm.notebook import trange, tqdm
from PIL import Image

import torch
import torchvision 
import torch.nn as nn

#from torch.cuda.amp import GradScaler
#from torch.cuda.amp import autocast

from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

num_workers = 4 if cuda else 0

print("Cuda = ", str(cuda), " with num_workers = ", str(num_workers),  " system version = ", sys.version)

Cuda =  True  with num_workers =  4  system version =  3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]


# 1. PairWise DataLoader


# 2. Data Loading Example


In [27]:
np.random.seed(0)
df = pd.read_csv("/content/gdrive/MyDrive/pair_out.csv")
print(df.shape[0])
df['split'] = np.random.randn(df.shape[0], 1)
df = df.iloc[:, 1:]
train_msk = df['split'] <= 0.7
val_msk = (df['split'] > 0.7) & (df['split'] <= 0.85)
test_msk = df['split'] > 0.85
train_data = df[train_msk]
val_data = df[val_msk]
test_data = df[test_msk]


119593


In [28]:
train_data = train_data.drop('split', 1).reset_index().drop('index',1)
val_data = val_data.drop('split',1).reset_index().drop('index',1)
test_data = test_data.drop('split',1).reset_index().drop('index',1)

In [29]:
# Define dataset class


class MyDataSet(Dataset):

    # load the dataset
    def __init__(self, data, **kwargs):
        self.X = data["frames"]
        self.Y = data["label"]

    # get number of items/rows in dataset
    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, index):
        x, y = self.X[index], self.Y[index]
        for c in ["[","]",",","'"]:
          x = x.replace(c, "")
          y = y.replace(c,"")
        x = x.split(" ")
        y = y.split(" ")
        y = list(map(lambda x:int(x),y))
        y = torch.FloatTensor(y)
        images = []
        for img_file in x:
            img = Image.open(cur_dir + img_file)
            img = torchvision.transforms.ToTensor()(img)
            img = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])(img)
            images.append(img)
        x = torch.stack(images)
        return x, y

In [30]:
cur_dir = '/content/converted_224x224/'

In [31]:
batch_size = 32

# training data
train_set = MyDataSet(train_data)
train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size, num_workers=8)

# validation data
val_set = MyDataSet(val_data)
val_loader = DataLoader(val_set, shuffle=False, batch_size=batch_size, num_workers=8)
# test data
test_set = MyDataSet(test_data)
test_loader = DataLoader(test_set, shuffle=False, batch_size=batch_size, num_workers=8)

In [32]:
# Model 

#These libraries help to interact with the operating system and the runtime environment respectively
import os
import sys

#Model/Training related libraries
import numpy as np
import torch
import torch.nn as nn
from torch import autograd
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import time
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import transforms, datasets

#Dataloader libraries
from torch.utils.data import DataLoader, Dataset
from PIL import Image

In [33]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channel, out_channel,stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride, padding=1, bias=False, dilation = 1)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu1 = nn.ReLU(inplace = True)
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, stride=1, padding= 1, bias=False, dilation = 1)
        self.bn2 =  nn.BatchNorm2d(out_channel)
        if stride == 1:
            self.shortcut = nn.Identity()
        else:
            self.shortcut = nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=stride)
        self.relu2 = nn.ReLU(inplace = True)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)

        shortcut = self.shortcut(x) 
        out = self.relu2(out + shortcut)
        
        return out

In [15]:
# Instead of using loop
# use slicing:   tesor[index. ..]
# x.squeeze(): squeeze to a vector; remove dimesion;
# x.unsqueeze(1); turns to (#,1)
# tensor.t() only applys to a two dimension tensor
# tensor.transpose applys to all dimesions;
# tensor.transpose(0,1) which dimension to switch
# view tensor won't create a copy; use the view to change the data; (only continous)
# x = x.view(3,4) print(x.is_continous); if you use transpose before, then the x.view won't work becasue view
# is not a contious action
# reshape tensor; (continous and noncontinous)
# x.shape
# check wheter a tensor is continous or not
# is_contiguous() check
# tensor.contiguous()  regturn a contiguous
# tensor.is_contiguous()
#print(tensor)
#print(tensor.stride()) whether the memory is left to right; top to bottom;

# unsqueeze can adding dimensions of 1;

# permute changes the order; transpose is a special case of permute


In [16]:
# create a tensor
# x.requires_grad_(True); will create a computational graph
# x.grad  


In [17]:
# slicing & torch.cat

In [34]:
class ClassificationNetwork(nn.Module):
    def __init__(self, in_features, num_classes,feat_dim = 512):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Conv2d(in_features, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1),
            ResidualBlock(64,64),
            ResidualBlock(64,64),
            ResidualBlock(64,64),
            ResidualBlock(64,128,stride = 2),
            ResidualBlock(128,128),
            ResidualBlock(128,128),
            ResidualBlock(128,128),
            ResidualBlock(128,256,stride = 2),
            ResidualBlock(256,256),
            ResidualBlock(256,256),
            ResidualBlock(256,256),
            ResidualBlock(256,256),
            ResidualBlock(256,256),
            ResidualBlock(256,512,stride = 2),
            ResidualBlock(512,512),
            ResidualBlock(512,512),
            nn.AdaptiveAvgPool2d((1, 1)), # For each channel, collapses (averages) the entire feature map (height & width) to 1x1
            nn.Flatten(1), # the above ends up with batch_size x 512 x 1 x 1, flatten to batch_size x 512
        )
        self.linear_emb = nn.Linear(512, feat_dim)
        self.relu = nn.ReLU(inplace=True)
        # output of CNN is b*5, 512 
        # before put into linear, has to reshape to x
        self.linear_output_1 = nn.Linear(1024,512)
        self.relu_output = nn.ReLU(inplace=True)
        self.dropout_output = nn.Dropout(p=0.4)
        self.linear_output_2 = nn.Linear(512,num_classes)
        self.sigm = nn.Sigmoid()

        for m in self.modules():
          if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
          elif isinstance(m, (nn.BatchNorm2d)):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0) 

        for m in self.modules():
          if isinstance(m, ResidualBlock):
            nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]  

    def forward(self, x, return_embedding=False):
        embedding = self.layers(x)
        embedding_out = self.linear_emb(embedding)
        embedding_out = self.relu(embedding_out)

        # output of CNN is b*5, 512 
        # before put into linear, has to reshape to x
        embedding.requires_grad_(True)
        embedding_reshape = torch.reshape(embedding, (-1,5,512))
        # embedding shape is b * 5 * 512
        # slicing and stack
        final_embeddings = []
        B,I,_ = embedding_reshape.shape
        for b in range(B):
          batch = embedding_reshape[b,:,:]
          batch_pair_embeddings = []
          for i in range(I):
            for j in range(i+1, I):
              pair_embedding_1 = batch[i,:]
              pair_embedding_2 = batch[j,:]
              pairs = torch.cat((pair_embedding_1,pair_embedding_2),dim=0)
              batch_pair_embeddings.append(pairs)
          paired_embeddings = torch.stack(batch_pair_embeddings)
          final_embeddings.append(paired_embeddings)
        # batch * 10 , 1024
        paris_embdeddings = torch.stack(final_embeddings).reshape(-1,1024) 
        output = self.linear_output_1(paris_embdeddings)
        output = self.relu_output(output)
        output = self.dropout_output(output)
        # output dimension is b*10, 1
        output = self.linear_output_2(output)
        output = self.sigm(output)

        if return_embedding:
            return embedding_out,output
        else:
            return output 

In [35]:
class ConstomLoss(nn.Module):
    def __init__(self):
        super(ConstomLoss, self).__init__()
 
    def forward(self, out, targets):
      # out shape (b*10, 1)
      # target shape batch * 10
      targets = targets.reshape(-1,1)
      batch, pairs = targets.shape
      loss = torch.nn.BCELoss()(out, targets)
      return loss    

In [36]:
# Train the model

def train_model(train_loader,train_dataset,model,criterion, optimizer):
    training_loss = 0
    accuracy = 0.0
    # Set model in 'Training mode'
    model.train()
    start_time = time.time()
    with tqdm(train_loader) as t:
      for i, (x, y) in enumerate(train_loader):
        t.set_description("Training")
        # clear the gradients
        optimizer.zero_grad()
        x.requires_grad_(True)
        # all data & model on same device
        x = torch.reshape(x, (-1,3, 224, 224))
        x = x.to(device)
        y = y.to(device)
        # compute the model output
        out = model(x)
        # y shape: batch * 10
        # calculate loss
        loss = criterion(out, y)
        # call scalar loss
        # Backward pass
        loss.backward()
        # Update model weights 
        optimizer.step()
       

        #caculate accuracy
        p = torch.reshape(out, (-1, 10) )
        predict = (p > 0.5) * (torch.ones(p.shape).to(device))
        accuracy += (predict == y).sum().item()
        
        training_loss += loss.item()

    end_time = time.time()
    acc = accuracy/(len(train_dataset)*10)
    training_loss /= len(train_loader)
    print('Training Loss: ', training_loss, 'Time: ',end_time - start_time, 's')
    print("Training accuracy: ", acc)
    torch.cuda.empty_cache()
    del x
    del y
    del loss
    return training_loss,acc

In [37]:
# Evaluate the model

def evaluate_model(val_loader, val_dataset, model,criterion):
  with torch.no_grad():
    model.eval()
    val_loss = 0.0
    accuracy = 0
    # Set model in validation mode
    
    with tqdm(val_loader) as t:
      for i, (x, y) in enumerate(val_loader):
        t.set_description("Evaluating")
         # all data & model on same device
        x = torch.reshape(x, (-1,3, 224, 224))
        x = x.to(device)
        y = y.to(device)
        # evaluate the model on the validation set
        out = model(x)  
        # Calculate validation loss
        # reshape the result from (b*10, 1) to b * 10
        # calculate loss
        loss = criterion(out, y)
  
        #caculate accuracy
        p = torch.reshape(out, (-1, 10) )
        predict = (p > 0.5) * (torch.ones(p.shape).to(device))
        accuracy += (predict == y).sum().item()
        
        val_loss += loss.item()
    
    # Calculate validation accuracy
    acc = accuracy/(len(val_dataset)*10)
    # calculate val loss
    val_loss /= len(val_loader)
    return val_loss,acc


In [38]:
numEpochs = 80
in_features = 3 # RGB channels
learningRate = 1e-2
weightDecay = 1e-4
num_classes = 1 # only 0 or 1


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ClassificationNetwork(in_features, num_classes)
#model = model.to(device)
#print(model)
# Define customized Loss function
criterion = ConstomLoss()

# Adam
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate, weight_decay=weightDecay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

In [23]:
base_acc = 0.0

# Define number of epochs
for epoch in range(numEpochs):
    
    # Train
    training_loss, train_acc= train_model(train_loader,train_set,model,criterion, optimizer)
    
    # Validation for validation dataset
    val_loss,val_acc = evaluate_model(val_loader,val_set,model,criterion)

    # scheduler
    scheduler.step(val_loss)
    print("Epoch: ",epoch+1,", current lr:", optimizer.state_dict()['param_groups'][0]['lr'])

    if val_acc > base_acc:
      model_save_name = 'shot_detection.pt'
      path = F"/content/gdrive/My Drive/{model_save_name}"
      print("Epoch: "+str(epoch+1)+",saved model parameter") 
      torch.save(model.state_dict(), path)
      base_acc = val_acc
    
    # Print log of training & validate accuracy
    print("Epoch: "+str(epoch+1)+", Training accuracy: "+str(train_acc*100)+"%, Validation accuracy:"+str(val_acc*100)+"%, Training loss: "+str(training_loss)+", Validation loss:"+str( val_loss))

Training:   0%|          | 0/2834 [22:36<?, ?it/s]


Training Loss:  0.6964791181984298 Time:  1356.727807044983 s
Training accuracy:  0.4995776123518059


Evaluating:   0%|          | 0/166 [00:30<?, ?it/s]


Epoch:  1 , current lr: 0.01
Epoch: 1,saved model parameter
Epoch: 1, Training accuracy: 49.95776123518059%, Validation accuracy:50.797128825085004%, Training loss: 0.6964791181984298, Validation loss:0.6930674206779663


Training:   0%|          | 0/2834 [22:36<?, ?it/s]


...

Training Loss:  0.6931775203244149 Time:  1357.435725927353 s
Training accuracy:  0.5006043562172594


Evaluating:   0%|          | 0/166 [00:30<?, ?it/s]


Epoch:  15 , current lr: 0.00125
Epoch: 15, Training accuracy: 50.060435621725944%, Validation accuracy:49.202871174914996%, Training loss: 0.6931775203244149, Validation loss:0.6932850322091436


Training:   0%|          | 0/2834 [11:49<?, ?it/s]


KeyboardInterrupt: ignored

In [39]:
# load model

print("Loading model")
model_save_name = 'shot_detection.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
model_dict = torch.load(path)
model.load_state_dict(model_dict)
model = model.to(device)
print(model)

Loading model
ClassificationNetwork(
  (layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Identity()
      (relu2): ReLU(inplace=True)
    )
    (5): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-

In [42]:
torch.cuda.empty_cache()
with torch.no_grad():
  model.eval()
  accuracy = 0
  with tqdm(test_loader) as t:
    for i, (x, y) in enumerate(test_loader):
      t.set_description("Testing")
      # all data & model on same device
      x = torch.reshape(x, (-1,3, 224, 224))
      x = x.to(device)
      y = y.to(device)
      # evaluate the model on the validation set
      out = model(x)  
      #caculate accuracy
      p = torch.reshape(out, (-1, 10) )
      predict = (p > 0.5) * (torch.ones(p.shape).to(device))
      accuracy += (predict == y).sum().item()

    test_acc = accuracy/ (len(test_set)*10)
    print(test_acc)


Testing:   0%|          | 0/739 [02:22<?, ?it/s]

0.5015492719268541
